In [5]:
import time
import requests
import dotenv
from VideoClass import *

BV = "BV1k24y197KC"
bili_key = dotenv.get_key(".env", "BILI_KEY")
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ko;q=0.5",
}


爬虫获取视频json

In [6]:
response = requests.get("https://bili.zhouql.vip/meta/" + BV, headers = header)
response.encoding = 'utf-8'
j = response.json()
j

{'code': 0,
 'message': '0',
 'ttl': 1,
 'data': {'bvid': 'BV1k24y197KC',
  'aid': 688879425,
  'videos': 32,
  'tid': 231,
  'tname': '计算机技术',
  'copyright': 1,
  'pic': 'http://i2.hdslb.com/bfs/archive/a0f11a138432271d0ce40b9c6a96d577d25acf43.jpg',
  'title': 'Kubernetes入门90分钟精讲(合集)—可能是B站最简单的K8S教程',
  'pubdate': 1665808200,
  'ctime': 1665300987,
  'desc': '课件地址：https://www.yuque.com/wukong-zorrm/qdoy5p',
  'desc_v2': [{'raw_text': '课件地址：https://www.yuque.com/wukong-zorrm/qdoy5p',
    'type': 1,
    'biz_id': 0}],
  'state': 0,
  'duration': 5441,
  'rights': {'bp': 0,
   'elec': 0,
   'download': 1,
   'movie': 0,
   'pay': 0,
   'hd5': 0,
   'no_reprint': 1,
   'autoplay': 1,
   'ugc_pay': 0,
   'is_cooperation': 0,
   'ugc_pay_preview': 0,
   'no_background': 0,
   'clean_mode': 0,
   'is_stein_gate': 0,
   'is_360': 0,
   'no_share': 0,
   'arc_pay': 0,
   'free_watch': 0},
  'owner': {'mid': 356591580,
   'name': '一小时技术精讲',
   'face': 'https://i0.hdslb.com/bfs/face/cbf97bddf076c

获取视频信息，但是不添加下载链接

In [7]:
parts = []
for each in j["data"]["pages"]:
    new_part = VideoPart(
        cid = each["cid"],
        page = each["page"],
        part = each["part"],
        duration = each["duration"],
        dimension = each["dimension"],
        first_frame_url = each["first_frame"],
        download_url = None
    )
    parts.append(new_part)

for each in parts:
    print(each.__dict__)

{'cid': 883837831, 'page': 1, 'part': '课程导读', 'duration': 71, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i1.hdslb.com/bfs/storyff/n221107qn1yscu5oakmrbg39acuv20cs_firsti.jpg', 'download_url': None}
{'cid': 856795535, 'page': 2, 'part': '课程目录', 'duration': 28, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i0.hdslb.com/bfs/storyff/n2210090830jc73ogktusa195l625g0k_firsti.jpg', 'download_url': None}
{'cid': 856795622, 'page': 3, 'part': 'kubernetes简介', 'duration': 184, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i0.hdslb.com/bfs/storyff/n221009142rymeabe6jshhhxiqf2azzh_firsti.jpg', 'download_url': None}
{'cid': 856795680, 'page': 4, 'part': 'kubernetes架构', 'duration': 176, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i1.hdslb.com/bfs/storyff/n221009083834wdl9a8ibv1w7mzpktoe_firsti.jpg', 'download_url': None}
{'cid': 856798978,

获取视频合集信息

In [8]:
video_set = VideoSet(
        aid = j["data"]["aid"],
        bvid = j["data"]["bvid"],
        title = j["data"]["title"],
        part_num = j["data"]["videos"],
        parts = parts,
        cover_url = j["data"]["pic"],
)
video_set.__dict__

{'aid': 688879425,
 'bvid': 'BV1k24y197KC',
 'title': 'Kubernetes入门90分钟精讲(合集)—可能是B站最简单的K8S教程',
 'part_num': 32,
 'parts': [<VideoClass.VideoPart at 0x14eb9d8ad50>,
 'cover_url': 'http://i2.hdslb.com/bfs/archive/a0f11a138432271d0ce40b9c6a96d577d25acf43.jpg'}

获取视频下载链接

In [9]:
for each in video_set.parts:
    response = requests.get("https://bili.zhouql.vip/download/" + str(video_set.aid) + "/" + str(each.cid), headers = header)
    response.encoding = 'utf-8'
    j = response.json()
    each.download_url = j["data"]["durl"][0]["url"]
    time.sleep(3)
    print(each.__dict__)

{'cid': 883837831, 'page': 1, 'part': '课程导读', 'duration': 71, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i1.hdslb.com/bfs/storyff/n221107qn1yscu5oakmrbg39acuv20cs_firsti.jpg', 'download_url': 'https://cn-jsnt-ct-01-25.bilivideo.com/upgcxcode/31/78/883837831/883837831-1-16.mp4?e=ig8euxZM2rNcNbRVhwdVhwdlhWdVhwdVhoNvNC8BqJIzNbfq9rVEuxTEnE8L5F6VnEsSTx0vkX8fqJeYTj_lta53NCM=&uipk=5&nbs=1&deadline=1701519703&gen=playurlv2&os=bcache&oi=2043500563&trid=0000d742edc02c604008bb50a3fad609f775h&mid=0&platform=html5&upsig=b2f6d1a31318a6704075672c39cb9e26&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,mid,platform&cdnid=9933&bvc=vod&nettype=0&f=h_0_0&bw=12320&logo=80000000'}
{'cid': 856795535, 'page': 2, 'part': '课程目录', 'duration': 28, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i0.hdslb.com/bfs/storyff/n2210090830jc73ogktusa195l625g0k_firsti.jpg', 'download_url': 'https://cn-jsnt-ct-01-23.bilivideo.com/upgcxcode/35/55/85

In [10]:
VideoUtils.download_videoset(video_set)

ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check'))